<a href="https://colab.research.google.com/github/kjapan87/TweetsUnsupervisedModel/blob/main/Tweets_Unsupervised_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 29 20:19:39 2020

@author: Dell
"""
import tweepy
import re
import pickle
from tweepy import OAuthHandler

#defining consumer key, consumer secret key, access token, & access secret key
API_Key=" "
API_Secret=" "
Access_token=" "
Access_token_secret=" "

#this step is intializing tokens for clients authentication
auth = OAuthHandler(API_Key,API_Secret) #authetication of application
auth.set_access_token(Access_token,Access_token_secret) #user authentication

#defining the string
args=["japan"];
api_obj=tweepy.API(auth,timeout=10,wait_on_rate_limit=True)

#save the tweets in a list
List_Tweets=[]
query=args[0]
if len(args)==1:
  for status in tweepy.Cursor(api_obj.search,q=query+" -filter:retweets",lang="en",result_type="recent").items(500):
    List_Tweets.append(status.text)

#Loading Pre-Trained model
List_Tweets
with open("classifier.pickle","rb") as PTmodel:
    clf=pickle.load(PTmodel)
    
with open("tfidfmodel.pickle","rb") as TDmodel:
    tdif=pickle.load(TDmodel)

total_positive=0
total_negative=0

#pre-processing start
import nltk
import re
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
from nltk.corpus import stopwords

for tweets in List_Tweets:
    tweets=re.sub(r'http\S+'," ",tweets)
    tweets=re.sub(r'https\S+'," ",tweets)
    tweets=re.sub(r'@\S+',"user",tweets)
    tweets=re.sub("[^a-zA-Z]"," ",tweets)
    tweets=tweets.lower()
    tweets=tweets.split()
    stemming = PorterStemmer()
    tweets = [stemming.stem(word) for word in tweets if not word in set(stopwords.words("english"))]
    tweets=" ".join(tweets)
    
    sent = clf.predict(tdif.transform([tweets]).toarray())
    #attaching
    if sent[0] == 1:
        total_positive += 1
    #elif:
     #   total_neutral
    else:
        total_negative += 1

print(total_positive)
print(total_negative)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

obj1 = ["pstv","ngtv"] #category of bars is named from this sentence
yAxis = np.arange(len(obj1))#order of the category
plt.bar(yAxis,[total_positive,total_negative])
plt.xticks(yAxis,obj1)
plt.ylabel("number")
plt.title("positive & negative tweets")
    

    

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.18.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py